## Scratch and Testing Notebook

Current objective: improve diffusion!

______
## Imports and Data

In [2]:
import numpy as np
import unittest
import sklearn.metrics.pairwise
import sklearn.preprocessing
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import PIL
import time
import glob
import copy
import my_eval
import sklearn.decomposition
import query
import parameterised_diffusion

In [3]:
data = {}

NOTEBOOK_DIR = "/home/sean/Code/Pawsey/4. Clean"


data = {}

for data_subset in ["oldenburger", "sutton"]:
    subdir = "./data/" + data_subset
    data[data_subset] = {}

    for descriptor in os.listdir(subdir):

        if descriptor == "names":
            data[data_subset][descriptor] = {"ox" : {}, "par" : {}}
            for fname in os.listdir(subdir + "/" + descriptor):
                split_name = fname[:-4].split("-")
                dataset = split_name[0]
                if fname.endswith("y-names.npy"):
                    data[data_subset][descriptor][dataset]["y"] = np.load("./data/{}/{}/{}".format(data_subset, descriptor, fname))
                else:
                    difficulty = split_name[2]
                    data[data_subset][descriptor][dataset][difficulty] = np.load("./data/{}/{}/{}".format(data_subset, descriptor, fname))

        else:
            data[data_subset][descriptor] = {"ox" : {"E" : {}, "M" : {}, "H" :{}},
                                "par" : {"E" : {}, "M" : {}, "H" :{}}}
            for fname in os.listdir(subdir + "/" + descriptor):
                split_name = fname[:-4].split("-")
                if len(split_name) == 3:
                    pass
                else:
                    (_, xy, dataset, difficulty) = split_name
                    data[data_subset][descriptor][dataset][difficulty][xy] = np.load("./data/{}/{}/{}".format(data_subset, descriptor, fname))
        
        print("Loaded " + descriptor)

data["oldenburger"]["swin"]["ox"]["E"]["x"]

Loaded swin
Loaded vgg
Loaded resnet
Loaded names
Loaded vit
Loaded names
Loaded sift-10k
Loaded sift-1k


array([[ 0.4381919 , -1.1369115 , -0.49100572, ..., -0.27456677,
         0.38102797, -0.30554023],
       [-0.19804995,  0.02098738,  0.52111053, ...,  0.44540596,
         0.8620084 ,  0.18907186],
       [ 1.0216093 , -0.06300209, -0.06569103, ...,  0.02202551,
        -0.32440802,  0.3858102 ],
       ...,
       [ 0.74518114, -0.9655011 , -0.55623275, ..., -0.39560622,
         0.3983633 , -0.4672271 ],
       [ 0.4493655 , -0.97439206, -0.61376625, ..., -0.19914342,
         0.27447924, -0.3482531 ],
       [ 0.04259995,  0.09633142,  0.65417933, ...,  0.5438953 ,
         0.53027916,  0.03832415]], dtype=float32)

____
## Baseline query

In [4]:
q = data["oldenburger"]["swin"]["ox"]["E"]["y"]
t = data["oldenburger"]["swin"]["ox"]["E"]["x"]
q_n = data["oldenburger"]["names"]["ox"]["y"]
t_n = data["oldenburger"]["names"]["ox"]["E"]



In [11]:
ranks = query.return_ranks("basic", q, t)
my_eval.evaluate(ranks, q_n, t_n)

(0.8838282489082855,
 {1: 0.8857142857142857, 5: 0.7371428571428572, 10: 0.6728571428571429})

____
## Modify diffusion parameters


In [15]:
parameterised_diffusion.kq
parameterised_diffusion.kd

50